# Домашнее задание 2. Градиентный спуск

Задание выполнил(а):

    (впишите свои фамилию и имя)

В этом ноутбуке мы решим задачу регрессии тремя способами:

* с использованием точной (аналитической) формулы,

* через готовую функцию из библиотеки scikit-learn,

* с помощью собственной реализации метода градиентного спуска.

На практике почти всегда применяются библиотечные реализации — они быстрее, надежнее и удобнее. Однако понимание того, как базовая задача машинного обучения решается «изнутри», чрезвычайно полезно. Это позволит:

* лучше разбираться в ограничениях и особенностях методов;

* осознанно выбирать и настраивать алгоритмы;

* легче переходить к изучению более сложных моделей, включая нейронные сети, где градиентный спуск является основным инструментом оптимизации.

## Импорт библиотек, установка константных значений

In [ ]:
import pandas as pd
import numpy as np

from sklearn.datasets import make_regression, fetch_california_housing
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
RANDOM_STATE = 42

## Генерация данных

Мы сами создадим синтетический датасет для того чтобы со всех сторон "покрутить" градиентный спуск.

Функция ```make_regression``` из ```sklearn.datasets``` создает искусственный датасет для задачи линейной регрессии.

Чтобы результаты генерации были воспроизводимыми, фиксируется зерно ```(RANDOM_STATE)```. Это гарантирует, что при каждом запуске будут получаться одинаковые данные. Не убирате и не меняйте его.

In [ ]:
np.random.seed(RANDOM_STATE)

X, y, _ = make_regression(n_samples=100000,              # число объектов
                          n_features=9,                  # число признаков
                          n_informative=8,               # число информативных признаков (остальные будут "шумовыми")
                          noise=100,                     # уровень шума в данных
                          coef=True,                     # значение True используется при генерации данных
                          random_state=RANDOM_STATE)

X = pd.DataFrame(data=X, columns=np.arange(0, X.shape[1]))
X[9] = X[6] + X[7] + np.random.random()*0.01

In [ ]:
X.shape, y.shape

((100000, 10), (100000,))

Сгенерировали датасет из 100000 объектов и 10 признаков у каждого.

Обучим модель линейной регрессии:

$$
a(x) = \beta_1 d_{1} + \beta_2 d_{2} + \beta_3 d_{3} + \beta_4 d_{4} + \beta_5 d_{5} + \beta_6 d_{6} + \beta_7 d_{7} + \beta_8 d_{8} + \beta_9 d_{9} + \beta_{10} d_{10} + \beta_0
$$

Которая минимизирует MSE:

$$
Q(a(X), Y) = \sum_i^{100000} (a(x_i) - y_i)^2
$$

## Практика

Реализуем метод градиентного спуска для обучения линейной регрессии.

### Задание 1 (10 баллов)


Напишите функцию, вычисляющую значение весов в линейной регрессии по точной (аналитически найденной) формуле:

$$w = (X^TX)^{-1}X^Ty$$

Комментарий: для поиска решения в векторном виде сначала необходимо добавить единичный столбец к матрице $X$.
Допишите для этого код ниже

In [ ]:
def ols_solution(X, y):
    X = np.hstack((np.ones((X.shape[0], 1)), X))
    # ваш код здесь
    w = np.linalg.inv(X.T @ X) @ X.T @ y


### Задание 2 (10 баллов)

Заполните функцию для предсказания модели по формуле
$$a(X)=Xw$$

In [ ]:
def prediction(X, w):
    X = np.hstack((np.ones((X.shape[0], 1)), X))
    # ваш код здесь
    predict = X @ w
    return predict


### Задание 3 (10 баллов)

Обучите коэффициенты линейной регрессии с помощью библиотеки <a href="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html"> **sklearn** </a>

Отдельно выведите оценку свободного коэффициента  ($\beta_0$ при $d_0 = 1$).

In [ ]:
model = LinearRegression()

# ваш код здесь
coefficients = model.coef_
intercept = model.intercept_

print("Коэффициенты (веса):", coefficients)
print("Оценка свободного коэффициента (β0):", intercept)

### Задание 4 (10 баллов)

Напишите функцию для вычисления среднеквадратичной ошибки

$MSE = \frac{1}{m}||Xw - y||^2_2$.

Здесь квадратичная ошибка записана в матричном виде, т.е. $X$ - матрица объект-признак, $w$ - вектор весов модели.
*  $Xw$ - вектор предсказания модели
*  $y$ - вектор правильных ответов,
и квадратичная ошибка - это квадрат нормы разности вектора предсказания и вектора правильных ответов.

Вычислить норму вектора в Python можно разными способами.  
Для данного задания воспользуйтесь готовой функцией из библиотеку numpy - `numpy.linalg.norm`.

In [ ]:
def compute_cost(X, y, theta):
    m = len(y)
    # ваш код здесь
    cost = np.linalg.norm(prediction - y) ** 2 / m
    return cost

### Задание 5 (30 баллов)

Реализуем градиентный спуск по формуле

$$w_{new} = w_{prev} - \nabla_w Q(w_{prev})$$

Вычислим градиент MSE:
$$\nabla_w Q(w)=\frac2m X^T(Xw-y).$$

Итак, реализуем метод градиентного спуска:

*  первым шагом добавим к матрице `X` единичный столбец - это константный признак, равный 1 на всех объектах.  
Он нужен, чтобы записать предсказание линейной регрессии в виде скалярного произведения и тем самым избавиться от знака суммы:
$a(x)=w_0+w_1x_1+...+w_dx_d=w_1\cdot 1+w_1x_1+...w_dx_d=(w,x)$  
В python скалярное произведение можно записать так: `w@x`

*  затем инициализируем случайным образом вектор весов `params`

*  зададим пустой массив `cost_track`, в который будем записывать ошибку на каждой итерации

*  наконец, в цикле по количеству эпох (итераций) будем обновлять веса по формуле градиентного спуска

In [ ]:
def gradient_descent(X, y, learning_rate, iterations):

    X = np.hstack((np.ones((X.shape[0], 1)), X)) # добавляем к Х столбец из 1
    params = ... # ваш код здесь

    m = X.shape[0]

    cost_track = ... # ваш код здесь

    for i in range(iterations):
        params = ... # ваш код здесь
        cost_track[i] = ... # ваш код здесь

    return cost_track, params

### Задание 6 (10 баллов)

Перепешите метод `gradient_descent`, используя критерий останова на ваш выбор

In [ ]:
def gradient_descent(X, y, learning_rate, iterations, epsilon):
  ...

### Задание 7 (20 баллов)

- Обучите линейную регрессию тремя методами (по точной формуле, с помощью функции из библиотеки `sklearn` и с помощью GD) на данных для задачи регрессии ($X, y$). Для GD используйте `learning_rate = 0.01, iterations = 10000`, `epsilon = 0.001`.

- С помощью каждого метода сделайте предсказание (на всех данных), вычислите качество предсказания r2 (`from sklearn.metrics import r2_score`). Для получения предсказания использоуйте функцию `predict`.


In [ ]:
# **План**

# 1 - находим веса одним из методов

# 2 - применяем функцию prediction для получения предсказаний с найденными весами

# 3 - вычисляем значение метрики r2